<a href="https://colab.research.google.com/github/kywch/geo-colab/blob/master/TreePedia_Step2_checking_gsv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**To access the google streetview API, one should have the API key.**
* First, get started with Google Maps platform: https://developers.google.com/maps/gmp-get-started
* Then, get an API key to use Street View Static API: https://developers.google.com/maps/documentation/streetview/get-api-key

---
This notebook uses google-streetview package: https://pypi.org/project/google-streetview/. 
* For documentation, please see https://rrwen.github.io/google_streetview/



In [1]:
!pip install google-streetview

  Created wheel for google-streetview: filename=google_streetview-1.2.9-cp36-none-any.whl size=9781 sha256=a23965f48dd5db81a072e78d86f4de8cfb2a6aca27752d128f16418af1ec8318
  Stored in directory: /root/.cache/pip/wheels/e1/ce/c0/e90d7ee251659d215b7ab21a7cd80cfd65e1720fc3dca3b9ed
  Created wheel for kwconfig: filename=kwconfig-1.1.7-cp36-none-any.whl size=4981 sha256=b45564c59a944b2ff1efff21c21205ead4fc32ba7743901da1080a840c3d50ee
  Stored in directory: /root/.cache/pip/wheels/22/fa/82/fed4c9e2829a6210470b121673adddedf169b6319112b3ba20
Successfully built google-streetview kwconfig


In [2]:
import pandas as pd
import google_streetview.api
import math

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!ls drive/My\ Drive/StreetPsych

Brookline_MA  Chicago  HydePark  key.txt


In [6]:
# Google Street View Static API key
with open('drive/My Drive/StreetPsych/key.txt') as file:
  api_key = file.read()

In [7]:
#location = 'HydePark'
#location = 'Chicago'
location = 'Brookline_MA'

picked_df = pd.read_csv('drive/My Drive/StreetPsych/' + location + '/seed_' + location + '.csv')
picked_df

,Unnamed: 0,geometry,bearing,osm_index,lat,lng
0,0,POINT (-71.12575351466042 42.32773786609903),233.914886,0,42.327738,-71.125754
1,1,POINT (-71.14738155640359 42.3089787289133),111.039266,1,42.308979,-71.147382
2,2,POINT (-71.146599487386 42.30837706991422),333.033300,3,42.308377,-71.146599
3,3,POINT (-71.16451937317942 42.3128648955299),285.269836,21,42.312865,-71.164519
4,4,POINT (-71.16502827573623 42.31303026362506),303.632335,21,42.313030,-71.165028
...,...,...,...,...,...,...
1405,1405,POINT (-71.115995784234 42.33280120281265),60.352672,3264,42.332801,-71.115996
1406,1406,POINT (-71.11550144962152 42.3330091966836),60.352591,3264,42.333009,-71.115501
1407,1407,POINT (-71.15045862781058 42.3008962719038),259.672377,3265,42.300896,-71.150459
1408,1408,POINT (-71.12574543690265 42.33212974943358),117.124299,3304,42.332130,-71.125745


In [8]:
params = []
for row in picked_df.head(20).iterrows():
  params.append({
            'key': api_key,
            'location': str(row[1]['lat']) + ',' + str(row[1]['lng'])
  })
#'size': '600x480', # max 640x640 pixels
#print(params)

In [9]:
results = google_streetview.api.results(params)
#results.preview()
len(results.metadata)

20

In [10]:
# input - df: a Dataframe, chunkSize: the chunk size
# output - a list of DataFrame
# purpose - splits the DataFrame into smaller chunks
def split_dataframe(df, chunk_size = 200): 
    chunks = list()
    num_chunks = len(df) // chunk_size + 1
    for i in range(num_chunks):
        chunks.append(df[i*chunk_size:(i+1)*chunk_size])
    return chunks

def generate_param(df, api_key):
  params = []
  for row in df.iterrows():
    params.append({
              'key': api_key,
              'location': str(row[1]['lat']) + ',' + str(row[1]['lng'])
    })
  return params

In [ ]:
#split_dataframe(picked_df)[1][['lat','lng']].reset_index(drop=True)

In [11]:
results_df = pd.DataFrame()
split_dfs = split_dataframe(picked_df)
count = 1

for df in split_dfs:
  print('Processing ', count, ' / ', len(split_dfs))
  results = google_streetview.api.results(generate_param(df, api_key))
  gsv_info = []
  for row in results.metadata:
    if 'OK' in row['status']:
      if 'Google' in row['copyright']:
        gsv_info.append([row['location']['lat'], row['location']['lng'], row['pano_id'], row['date'], row['status']])
      else:
        gsv_info.append([math.nan, math.nan, math.nan, math.nan, math.nan])
    else:
      gsv_info.append([math.nan, math.nan, math.nan, math.nan, math.nan])

  results_df = results_df.append( pd.DataFrame(gsv_info, columns = ['gsv_lat', 'gsv_lng', 'pano_id', 'date', 'status']) )
  count += 1

print('Done.')  


Processing  1  /  8
Processing  2  /  8
Processing  3  /  8
Processing  4  /  8
Processing  5  /  8
Processing  6  /  8
Processing  7  /  8
Processing  8  /  8
Done.


In [12]:
combined_df = picked_df[['lat','lng','bearing','osm_index']].merge(results_df.reset_index(drop=True), left_index=True, right_index=True, suffixes=(False, False))
combined_df  

,lat,lng,bearing,osm_index,gsv_lat,gsv_lng,pano_id,date,status
0,42.327738,-71.125754,233.914886,0,42.327726,-71.125726,qPFd8ZiCTDtFRbIuXrD_fQ,2018-06,OK
1,42.308979,-71.147382,111.039266,1,NaN,NaN,NaN,NaN,NaN
2,42.308377,-71.146599,333.033300,3,NaN,NaN,NaN,NaN,NaN
3,42.312865,-71.164519,285.269836,21,42.312834,-71.164465,_6Zd2vQL1HJBJ0Rc3yvxNA,2011-07,OK
4,42.313030,-71.165028,303.632335,21,42.312999,-71.164982,ACStUzQrXsKtXm-CR17cTw,2011-07,OK
...,...,...,...,...,...,...,...,...,...
1405,42.332801,-71.115996,60.352672,3264,42.332802,-71.116007,Dh9jLDcWVcrIbrI5eGkneA,2018-05,OK
1406,42.333009,-71.115501,60.352591,3264,42.333028,-71.115462,DUW77JTp8KB6pLnvqGPYdA,2018-05,OK
1407,42.300896,-71.150459,259.672377,3265,42.300895,-71.150479,IGjcwamebot1aOxpFalugg,2016-06,OK
1408,42.332130,-71.125745,117.124299,3304,42.332129,-71.125785,8oBKpVMlNAy9dGh9h0azqg,2019-08,OK


In [13]:
combined_df.to_csv('drive/My Drive/StreetPsych/' + location + '/gsv_seed_' + location + '.csv')

In [18]:
sum(combined_df['date'] > '2018')

1067